In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import pandas as pd
from PyPDF2 import PdfFileReader
from io import BytesIO
from snowflake.snowpark import types as T
from snowflake.snowpark.files import SnowflakeFile
from snowflake.snowpark.types import StringType
from snowflake.snowpark.types import StringType, StructField, StructType
import streamlit as st

from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
st.image("https://raw.githubusercontent.com/cromano8/RAG_Snowflake/main/images/RAG_Arch.png")

In [11]:
def readpdf(file_path):
    whole_text = ""
    with SnowflakeFile.open(file_path, 'rb') as file:
        f = BytesIO(file.readall())
        pdf_reader = PdfFileReader(f)
        whole_text = ""
        for page in pdf_reader.pages:
            whole_text += page.extract_text()
    return whole_text

In [12]:
#Register the UDF. 
session.udf.register(
    func = readpdf
  , return_type = StringType()
  , input_types = [StringType()]
  , is_permanent = True
  , name = 'SNOWPARK_PDF'
  , replace = True
  , packages=['snowflake-snowpark-python','pypdf2','PyCryptodome']
  , stage_location = 'RAG'
)

In [ ]:
st.image( "https://raw.githubusercontent.com/cromano8/RAG_Snowflake/main/images/Extract_Text.png" )

In [15]:
CREATE OR REPLACE TABLE RAW_TEXT AS
SELECT
    relative_path
    , file_url
    , snowpark_pdf(build_scoped_file_url(@RAG, relative_path)) as raw_text
from directory(@RAG)

[Row(status='Table RAW_TEXT successfully created.')]

In [16]:
select * from RAW_TEXT;

---------------------------------------------------------------------------------------------------------------------------------
|"RELATIVE_PATH"      |"FILE_URL"                                          |"RAW_TEXT"                                          |
---------------------------------------------------------------------------------------------------------------------------------
|Grant_and_Grant.pdf  |https://xwb80589.snowflakecomputing.com/api/fil...  |Evolution of CharacterDisplacement in Darwin’s ...  |
|                     |                                                    |Peter R. Grant*and B. Rosemary GrantCompetitor ...  |
|                     |                                                    |Character displacement (1,2) is an evolu-tionar...  |
|                     |                                                    |2) has been referredto as the classical case of...  |
|                     |                                                    |Department of 

In [ ]:
--Optional : This will fail due to tokens exceeding limit, which means we need to chunk!
SELECT
SNOWFLAKE.CORTEX.COMPLETE('llama2-70b-chat',CONCAT('summarise the following text',raw_text)) 
FROM
RAW_TEXT
LIMIT 1;

A note on chunking
-----
Chunking is the process of splitting a large body of text into smaller 'chunks' whilst attempting to keep as much relevant information as possible. Make the chunks too small and you run the risk of removing key information that the model requires to answer the question. Too large and it may be harder to retreive the correct body of text from the vector search - or spend tokens excessively.

There are many strategies towards chunking. Eg - pass the most relevant, top n relevant chunks, or pass the most relevent chunk + the chunk either side of that one. Play around and see what works for your use case!

In [ ]:
st.image( "https://raw.githubusercontent.com/cromano8/RAG_Snowflake/main/images/Chunk.text.png" )

In [17]:
class text_chunker:

    def process(self,text):        
        text_raw=[]
        text_raw.append(text) 
        
        text_splitter = RecursiveCharacterTextSplitter(
            separators = ["\n"], # Define an appropriate separator. New line is good typically!
            chunk_size = 1000, #Adjust this as you see fit
            chunk_overlap  = 100, #This let's text have some form of overlap. Useful for keeping chunks contextual
            length_function = len,
            add_start_index = True #Optional but useful if you'd like to feed the chunk before/after
        )
    
        chunks = text_splitter.create_documents(text_raw)
        df = pd.DataFrame(chunks, columns=['chunks','meta'])
        
        yield from df.itertuples(index=False, name=None)

In [18]:
#Register the UDTF

schema = StructType([
     StructField("chunk", StringType()),
    StructField("meta", StringType()),
 ])

session.udtf.register( 
    handler = text_chunker,
    output_schema= schema, 
    input_types = [StringType()] , 
    is_permanent = True , 
    name = 'CHUNK_TEXT' , 
    replace = True , 
    packages=['pandas','langchain'], stage_location = 'RAG')

In [20]:
CREATE OR REPLACE TABLE CHUNK_TEXT AS
SELECT
        relative_path,
        func.*
    FROM raw_text AS raw,
         TABLE(chunk_text(raw_text)) as func;

[Row(status='Table CHUNK_TEXT successfully created.')]

In [ ]:
select * from chunk_text;

In [ ]:
ALTER TABLE chunk_text ADD COLUMN vec VECTOR(FLOAT, 768);

In [ ]:
st.image( "https://raw.githubusercontent.com/cromano8/RAG_Snowflake/main/images/Embed_text.png" )

In [ ]:
st.image( "https://raw.githubusercontent.com/cromano8/RAG_Snowflake/main/images/embedding_functions.png" )

In [ ]:
UPDATE chunk_text SET vec = SNOWFLAKE.CORTEX.EMBED_TEXT('e5-base-v2', chunk);

In [ ]:
select * from chunk_text
limit 5;

In [ ]:
select snowflake.cortex.complete('mistral-7b','What percent of snowflake customers process unstructured data?') as mixtral_respone;

In [ ]:
st.image( "https://raw.githubusercontent.com/cromano8/RAG_Snowflake/main/images/rag_query.png" )

In [ ]:
select relative_path,chunk, vector_l2_distance(
            snowflake.cortex.embed_text('e5-base-v2', 
            'What % of snowflake customers process unstructured data?'
            ), vec
            ) as distance from chunk_text 
            order by distance limit 5;

In [ ]:
select relative_path,chunk, vector_l2_distance(
            snowflake.cortex.embed_text('e5-base-v2', 
            'What data does the carolina panthers collect about me?'
            ), vec
            ) as distance from chunk_text 
            order by distance limit 5;

In [ ]:
select snowflake.cortex.complete(
    'mistral-7b', 
    concat( 
        'Answer the question based on the context. Be concise.','Context: ',
        (select array_agg(*)::varchar from (
            (select chunk from chunk_text 
            order by vector_l2_distance(
            snowflake.cortex.embed_text('e5-base-v2', 
            'What % of snowflake customers process unstructured data?'
            ), vec
            ) limit 5))
            ),
        'Question: ', 
        'What % of snowflake customers process unstructured data?',
        'Answer: '
    )
) as response;

In [ ]:
st.image( "https://raw.githubusercontent.com/cromano8/RAG_Snowflake/main/images/sis.png" )

In [ ]:
st.title("Ask Your Data Anything :snowflake:")
st.write("""Built using end-to-end RAG in Snowflake with Cortex functions.""")

docs = session.table('chunk_text').select('RELATIVE_PATH').distinct().collect()
doc = st.selectbox('Select you document', docs)

model = st.selectbox('Select your model:',('mistral-large','mistral-7b','mixtral-8x7b','llama2-chat-70b','gemma-7b'))

prompt = st.text_input("Enter prompt", placeholder="What % of customers use Snowflake to process unstructured data?", label_visibility="collapsed")

quest_q = f'''
SELECT SNOWFLAKE.CORTEX.COMPLETE(
    '{model}',
        'In 2 sentences or less, you are a pilot flying a 1967 cessna 172h answer the following question: - {prompt}'
) as response
'''

quest_q2 = f'''
select snowflake.cortex.complete(
    '{model}', 
    concat( 
        'Answer the question based on the context. Be concise.','Context: ',
        (select array_agg(*)::varchar from (
            (select chunk from chunk_text 
            where RELATIVE_PATH = '{doc}'
            order by vector_l2_distance(
            snowflake.cortex.embed_text('e5-base-v2', 
            '{prompt}'
            ), vec
            ) limit 5))
            ),
        'Question: ', 
        '{prompt}',
        'Answer: '
    )
) as response;
'''

if prompt:
    df_query = session.sql(quest_q).to_pandas()
    st.header('Result without RAG and chunking')
    st.write(df_query['RESPONSE'][0])
    st.header('Result with RAG and chunking')
    df_query = session.sql(quest_q2).to_pandas()
    # df_query['PROMPT'] = prompt
    st.write(df_query['RESPONSE'][0])
    # df = session.create_dataframe(df_query)
    # df.write.save_as_table("RAG_INPUTS", mode="append")
    # df = session.table('RAG_INPUTS')
    # st.dataframe(df)